In [2]:
import os
import re
import shutil
import random

import pickle
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch import optim
import torch.nn.functional as F
from torchvision import models, transforms, datasets, utils
from tqdm import tqdm
from tqdm import tqdm_notebook
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import itertools

pd.set_option('display.max_columns', 1000)

%load_ext autoreload
%autoreload 2

/Users/orianecavrois/Documents/Perso/MachineLearning/Github/Protein-Specific-Drug-Generation/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Use available device

In [3]:
device = "cpu"
# if torch.cuda.is_available():
#     print("using gpu")
#     device = torch.device("gpu")
# elif torch.backends.mps.is_available():
#     print("using mps")
#     device = torch.device("mps")

In [4]:
df = pd.read_csv('preprocessed_data.csv')

In [5]:
df

,Unnamed: 0,BindingDB Reactant_set_id,Ligand SMILES,Ligand InChI,Ligand InChI Key,BindingDB MonomerID,BindingDB Ligand Name,Target Name,Target Source Organism According to Curator or DataSource,BindingDB Target Chain Sequence,Target Source Organism According to Curator or DataSource.1
0,1960,1961,Fc1ccc(cc1)C1(CC1)C(=O)N1CC2CC2(C1)c1c[nH]c2nc...,InChI=1S/C22H20FN3O/c23-16-5-3-14(4-6-16)21(7-...,NVEWYSISVYQERO-UHFFFAOYSA-N,50390707,"CHEMBL2070316::US8486964, 1",11-beta-hydroxysteroid dehydrogenase 1,Homo sapiens,MAFMKKYLLPILGLFMAYYYYSANEEFRPEMLQGKKVIVTGASKGI...,Homo sapiens
1,2879,2880,Cn1ncc2c(cccc12)-c1cccn2nc(Nc3ccc4CCN(CCc4c3)C...,"InChI=1S/C29H31N7O2/c1-29(2,3)38-28(37)35-15-1...",MAZKPYXDQCNDAZ-UHFFFAOYSA-N,2013,"US8501936, 298::US8501936, 300",Tyrosine-protein kinase JAK2,Homo sapiens,MGMACLTMTEMEGTSTSSIYQNGDISGNANSMKQIDPVLQVYLYHS...,Homo sapiens
2,3038,3039,Cn1ncc2c(cccc12)-c1cccn2nc(Nc3ccc4CCN(CCc4c3)C...,"InChI=1S/C29H31N7O2/c1-29(2,3)38-28(37)35-15-1...",MAZKPYXDQCNDAZ-UHFFFAOYSA-N,2013,"US8501936, 298::US8501936, 300",Tyrosine-protein kinase JAK2,Homo sapiens,MGMACLTMTEMEGTSTSSIYQNGDISGNANSMKQIDPVLQVYLYHS...,Homo sapiens
3,3129,3130,CN[C@@H]1C[C@H]2O[C@@](C)([C@@H]1OC)n1c3ccccc3...,InChI=1S/C28H26N4O3/c1-28-26(34-3)17(29-2)12-2...,HKSZLNNOFSGOKW-FYTWVXJKSA-N,2579,"(2S,3R,4R,6R)-3-methoxy-2-methyl-4-(methylamin...",Protein kinase C beta type,Rattus norvegicus,MADPAAGPPPSEGEESTVRFARKGALRQKNVHEVKNHKFTARFFKQ...,Rattus norvegicus
4,3322,3323,O=C1NC(=O)C(=C1c1c[nH]c2ccccc12)c1c[nH]c2ccccc12,InChI=1S/C20H13N3O2/c24-19-17(13-9-21-15-7-3-1...,DQYBRTASHMYDJG-UHFFFAOYSA-N,2583,"3,4-Bis(3-indolyl)-1H-pyrrole-2,5-dione::3,4-b...",cAMP-dependent protein kinase catalytic subuni...,Bos taurus,MGNAAAAKKGSEQESVKEFLAKAKEDFLKKWENPAQNTAHLDQFER...,Bos taurus
...,...,...,...,...,...,...,...,...,...,...,...
462033,2830480,51508286,C[C@@H]1c2ccc(cc2C(=CN(C)C1=O)c1cnn(C)c1)[C@@H...,InChI=1S/C18H21N3O3/c1-11-14-5-4-12(17(23)10-2...,CJVJSGBELMMESS-DIFFPNOSSA-N,50607768,CHEMBL5219602,C-C motif chemokine 2,Homo sapiens,MKVSAALLCLLLIAATFIPQGLAQPDAINAPVTCCYNFTNRKISVQ...,Homo sapiens
462034,2830481,51508287,C[C@@H]1c2ccc(OCCO)cc2C(=CN(C)C1=O)c1c[nH]c(C)n1,InChI=1S/C18H21N3O3/c1-11-14-5-4-13(24-7-6-22)...,JKHAONFZTKWYFI-LLVKDONJSA-N,50607769,CHEMBL5219538,C-C motif chemokine 2,Homo sapiens,MKVSAALLCLLLIAATFIPQGLAQPDAINAPVTCCYNFTNRKISVQ...,Homo sapiens
462035,2830482,51508288,C[C@@H]1c2ccc(cc2C(=CN(C)C1=O)c1cnn(C)c1)C(CO)CO,InChI=1S/C19H23N3O3/c1-12-16-5-4-13(15(10-23)1...,XRYCKSUJUPLGRF-GFCCVEGCSA-N,50607770,CHEMBL5219919,C-C motif chemokine 2,Homo sapiens,MKVSAALLCLLLIAATFIPQGLAQPDAINAPVTCCYNFTNRKISVQ...,Homo sapiens
462036,2830519,51508325,CN(CCOc1ccc(C[C@H](Nc2ccccc2C(=O)c2ccccc2)C(O)...,InChI=1S/C30H29N3O4/c1-33(28-13-7-8-18-31-28)1...,QTQMRBZOBKYXCG-MHZLTWQESA-N,50085048,(S)-2-(2-Benzoyl-phenylamino)-3-{4-[2-(methyl-...,Peroxisome proliferator-activated receptor gamma,Homo sapiens,MGETLGDSPIDPESDSFTDTLSANISQEMTMVDTEMPFWPTNFGIS...,Homo sapiens


In [6]:
df['BindingDB Target Chain Sequence']

0         MAFMKKYLLPILGLFMAYYYYSANEEFRPEMLQGKKVIVTGASKGI...
1         MGMACLTMTEMEGTSTSSIYQNGDISGNANSMKQIDPVLQVYLYHS...
2         MGMACLTMTEMEGTSTSSIYQNGDISGNANSMKQIDPVLQVYLYHS...
3         MADPAAGPPPSEGEESTVRFARKGALRQKNVHEVKNHKFTARFFKQ...
4         MGNAAAAKKGSEQESVKEFLAKAKEDFLKKWENPAQNTAHLDQFER...
                                ...                        
462033    MKVSAALLCLLLIAATFIPQGLAQPDAINAPVTCCYNFTNRKISVQ...
462034    MKVSAALLCLLLIAATFIPQGLAQPDAINAPVTCCYNFTNRKISVQ...
462035    MKVSAALLCLLLIAATFIPQGLAQPDAINAPVTCCYNFTNRKISVQ...
462036    MGETLGDSPIDPESDSFTDTLSANISQEMTMVDTEMPFWPTNFGIS...
462037    MDTKHFLPLDFSTQVNSSLTSPTGRGSMAAPSLHPSLGPGIGSPGQ...
Name: BindingDB Target Chain Sequence, Length: 462038, dtype: object

In [7]:
df['BindingDB Target Chain Sequence'].value_counts()

BindingDB Target Chain Sequence
MAAVILESIFLKRSQQKKKTSPLNFKKRLFLLTVHKLSYYEYDFERGRRGSKKGSIDVEKITCVETVVPEKNPPPERQIPRRGEESSEMEQISIIERFPYPFQVVYDEGPLYVFSPTEELRKRWIHQLKNVIRYNSDLVQKYHPCFWIDGQYLCCSQTAKNAMGCQILENRNGSLKPGSSHRKTKKPLPPTPEEDQILKKPLPPEPAAAPVSTSELKKVVALYDYMPMNANDLQLRKGDEYFILEESNLPWWRARDKNGQEGYIPSNYVTEAEDSIEMYEWYSKHMTRSQAEQLLKQEGKEGGFIVRDSSKAGKYTVSVFAKSTGDPQGVIRHYVVCSTPQSQYYLAEKHLFSTIPELINYHQHNSAGLISRLKYPVSQQNKNAPSTAGLGYGSWEIDPKDLTFLKELGTGQFGVVKYGKWRGQYDVAIKMIKEGSMSEDEFIEEAKVMMNLSHEKLVQLYGVCTKQRPIFIITEYMANGCLLNYLREMRHRFQTQQLLEMCKDVCEAMEYLESKQFLHRDLAARNCLVNDQGVVKVSDFGLSRYVLDDEYTSSVGSKFPVRWSPPEVLMYSKFSSKSDIWAFGVLMWEIYSLGKMPYERFTNSETAEHIAQGLRLYRPHLASEKVYTIMYSCWHEKADERPTFKILLSNILDVMDEES                                                                                                                                                                                                                                                                                                                     

In [8]:
df['BindingDB MonomerID'].value_counts()

BindingDB MonomerID
2579        1282
19149        693
4814         310
50308060     278
16673        258
            ... 
584319         1
584320         1
584321         1
584323         1
50607770       1
Name: count, Length: 278773, dtype: int64

In [9]:
df['Ligand SMILES'].value_counts()

Ligand SMILES
CN[C@@H]1C[C@H]2O[C@@](C)([C@@H]1OC)n1c3ccccc3c3c4CNC(=O)c4c4c5ccccc5n2c4c13                       1282
ONC(=O)CCCCCCC(=O)Nc1ccccc1                                                                         693
CCN(CC)CCNC(=O)c1c(C)[nH]c(\C=C2/C(=O)Nc3ccc(F)cc23)c1C                                             310
C[C@]12O[C@H](C[C@]1(O)CO)n1c3ccccc3c3c4C(=O)NCc4c4c5ccccc5n2c4c13                                  278
CNC(=O)c1cc(Oc2ccc(NC(=O)Nc3ccc(Cl)c(c3)C(F)(F)F)cc2)ccn1                                           258
                                                                                                   ... 
OCC(O)CCS(=O)(=O)NC(=O)c1ccc(cc1)N1C[C@@H]2C[C@H]1C[C@H]2OCc1c(onc1-c1c(Cl)cccc1Cl)C1CC1              1
OC1(CCCCC1)S(=O)(=O)NC(=O)c1ccc(cc1)N1C[C@@H]2C[C@H]1C[C@H]2OCc1c(onc1-c1c(Cl)cccc1Cl)C1CC1           1
OC1CCC(CC1)S(=O)(=O)NC(=O)c1ccc(cc1)N1C[C@@H]2C[C@H]1C[C@H]2OCc1c(onc1-c1c(Cl)cccc1Cl)C1CC1           1
CCCCCCCCCCCCS(=O)(=O)NC(=O)c1ccc(N2C[C@@H]3C[C@H]2

### Preprocess input

In [14]:
# Check if all sequences are uppercase
for value in df['BindingDB Target Chain Sequence'].values:
    if value.upper() != value:
        print(value)

In [15]:
df['BindingDB Target Chain Sequence'] = df['BindingDB Target Chain Sequence'].apply(lambda x: x.upper())

### Encode the acid amino and SMILES symbols

In [16]:
PAD_IDX = 0
SOS_IDX = 1
EOS_IDX = 2

PAD = '<pad>'
SOS = '<sos>'
EOS = '<eos>'

symb2index = {'<pad>':0,'<sos>':1,'<eos>':2} # < or 0 marks the Beginning of a sequence, > or 1 marks the end of a sequence
symb2count = {}
index2symb = {0:'<pad>',1:'<sos>',2:'<eos>'} # < or 0 marks the Beginning of a sequence, > or 1 marks the end of a sequence

for chain in tqdm(df['BindingDB Target Chain Sequence'].values):
    for symb in chain.upper():
        if symb not in symb2index.keys():
            symb2index[symb] = len(symb2index.keys())
            symb2count[symb] = 1
            index2symb[len(symb2index.keys()) - 1] = symb
        else:
            symb2count[symb] += 1

for chain in tqdm(df['Ligand SMILES'].values):
    for symb in chain:
        if symb not in symb2index.keys():
            symb2index[symb] = len(symb2index.keys())
            symb2count[symb] = 1
            index2symb[len(symb2index.keys()) - 1] = symb
        else:
            symb2count[symb] += 1


100%|██████████| 462038/462038 [00:03<00:00, 147032.07it/s]


In [17]:
symb2index

{'<pad>': 0,
 '<sos>': 1,
 '<eos>': 2,
 'M': 3,
 'A': 4,
 'F': 5,
 'K': 6,
 'Y': 7,
 'L': 8,
 'P': 9,
 'I': 10,
 'G': 11,
 'S': 12,
 'N': 13,
 'E': 14,
 'R': 15,
 'Q': 16,
 'V': 17,
 'T': 18,
 'H': 19,
 'C': 20,
 'D': 21,
 'W': 22,
 'c': 23,
 '1': 24,
 '(': 25,
 ')': 26,
 '=': 27,
 'O': 28,
 '2': 29,
 '[': 30,
 'n': 31,
 ']': 32,
 '-': 33,
 '3': 34,
 '4': 35,
 '@': 36,
 '5': 37,
 '+': 38,
 'l': 39,
 'B': 40,
 'r': 41,
 '#': 42,
 '\\': 43,
 '/': 44,
 's': 45,
 'o': 46,
 '6': 47,
 'b': 48,
 '.': 49,
 'u': 50,
 'i': 51,
 'e': 52,
 'a': 53,
 '7': 54,
 '8': 55,
 '9': 56,
 'd': 57,
 '*': 58,
 'Z': 59,
 'g': 60,
 '%': 61,
 '0': 62}

In [18]:
symb2count

{'M': 7736972,
 'A': 20145004,
 'F': 13111819,
 'K': 17778577,
 'Y': 10617616,
 'L': 31619214,
 'P': 17505178,
 'I': 15755803,
 'G': 19599324,
 'S': 23251366,
 'N': 13467865,
 'E': 20384665,
 'R': 16239316,
 'Q': 12711609,
 'V': 20280005,
 'T': 16138074,
 'H': 8506707,
 'C': 12029948,
 'D': 15278248,
 'W': 4428697,
 'c': 6530317,
 '1': 2081769,
 '(': 2681481,
 ')': 2681481,
 '=': 799485,
 'O': 1273819,
 '2': 1200594,
 '[': 502344,
 'n': 1132085,
 ']': 502344,
 '-': 284847,
 '3': 509095,
 '4': 145312,
 '@': 564283,
 '5': 27146,
 '+': 8247,
 'l': 123116,
 'B': 13916,
 'r': 12232,
 '#': 69068,
 '\\': 44321,
 '/': 12526,
 's': 51437,
 'o': 36387,
 '6': 2298,
 'b': 2,
 '.': 3241,
 'u': 32,
 'i': 262,
 'e': 196,
 'a': 165,
 '7': 300,
 '8': 84,
 '9': 60,
 'd': 5,
 '*': 3,
 'Z': 1,
 'g': 3,
 '%': 28,
 '0': 20}

In [19]:
index2symb

{0: '<pad>',
 1: '<sos>',
 2: '<eos>',
 3: 'M',
 4: 'A',
 5: 'F',
 6: 'K',
 7: 'Y',
 8: 'L',
 9: 'P',
 10: 'I',
 11: 'G',
 12: 'S',
 13: 'N',
 14: 'E',
 15: 'R',
 16: 'Q',
 17: 'V',
 18: 'T',
 19: 'H',
 20: 'C',
 21: 'D',
 22: 'W',
 23: 'c',
 24: '1',
 25: '(',
 26: ')',
 27: '=',
 28: 'O',
 29: '2',
 30: '[',
 31: 'n',
 32: ']',
 33: '-',
 34: '3',
 35: '4',
 36: '@',
 37: '5',
 38: '+',
 39: 'l',
 40: 'B',
 41: 'r',
 42: '#',
 43: '\\',
 44: '/',
 45: 's',
 46: 'o',
 47: '6',
 48: 'b',
 49: '.',
 50: 'u',
 51: 'i',
 52: 'e',
 53: 'a',
 54: '7',
 55: '8',
 56: '9',
 57: 'd',
 58: '*',
 59: 'Z',
 60: 'g',
 61: '%',
 62: '0'}

In [20]:
len(symb2index)

63

In [21]:
with open('symbole_to_index_encoding.pk', 'wb') as file:
    pickle.dump(symb2index, file)
    
with open('index_to_symbol_decoding.pk', 'wb') as file:
    pickle.dump(index2symb, file)

### Create tensors of (ligand, protein) pairs

In [22]:
pairs = []

for index in tqdm(df.index):
    pairs.append([df.loc[index,'BindingDB Target Chain Sequence'], df.loc[index,'Ligand SMILES']])

  0%|          | 0/462038 [00:00<?, ?it/s]

100%|██████████| 462038/462038 [00:03<00:00, 139922.10it/s]


In [23]:
def tensorsFromPair(pair):
    target = pair[0]
    ligand = pair[1]
    
    indexes_target = [symb2index[aa] for aa in target.upper()]
    indexes_ligand = [SOS_IDX] + [symb2index[aa] for aa in ligand] + [EOS_IDX]
    
    target_tensor = torch.tensor(indexes_target, dtype=torch.long, device=device).view(-1, 1).to("cpu")
    ligand_tensor = torch.tensor(indexes_ligand, dtype=torch.long, device=device).view(-1, 1).to("cpu")
    
    return (target_tensor, ligand_tensor)

In [24]:
# Split train and test on unique target chain sequence
target_train, target_test_val = train_test_split(df['BindingDB Target Chain Sequence'].unique(), test_size=0.3, random_state=42)
target_val, target_test = train_test_split(target_test_val, test_size=0.5, random_state=42)

In [25]:
pairs_train = [pair for pair in pairs if pair[0] in target_train]
pairs_test = [pair for pair in pairs if pair[0] in target_test]
pairs_val = [pair for pair in pairs if pair[0] in target_val]

In [26]:
len(pairs_train)

337459

In [27]:
len(pairs_test)

52514

In [28]:
len(pairs_val)

72065

In [29]:
pairs_test_tensor = [tensorsFromPair(pv) for pv in pairs_test]
pairs_val_tensor = [tensorsFromPair(pairs_val[pv]) for pv in np.random.choice(range(len(pairs_val)), size=len(pairs_val), replace=False)]
pairs_train_tensor = [tensorsFromPair(pairs_train[pv]) for pv in np.random.choice(range(len(pairs_train)), size=len(pairs_train), replace=False)]

In [30]:
pairs_train_tensor[0][1]

tensor([[ 1],
        [20],
        [28],
        [23],
        [24],
        [23],
        [23],
        [23],
        [23],
        [23],
        [24],
        [12],
        [23],
        [24],
        [23],
        [23],
        [23],
        [25],
        [43],
        [20],
        [27],
        [20],
        [43],
        [20],
        [25],
        [27],
        [28],
        [26],
        [13],
        [29],
        [20],
        [20],
        [20],
        [25],
        [20],
        [20],
        [29],
        [26],
        [20],
        [25],
        [28],
        [26],
        [27],
        [28],
        [26],
        [23],
        [25],
        [23],
        [24],
        [20],
        [25],
        [ 5],
        [26],
        [25],
        [ 5],
        [26],
        [ 5],
        [26],
        [20],
        [25],
        [ 5],
        [26],
        [25],
        [ 5],
        [26],
        [ 5],
        [ 2]])

In [31]:
print(pairs_train_tensor[0][0].device)

cpu


In [32]:
with open('pairs_train_tensors.pk', 'wb') as file:
    pickle.dump(pairs_train_tensor, file)
    
with open('pairs_test_tensors.pk', 'wb') as file:
    pickle.dump(pairs_test_tensor, file)
    
with open('pairs_val_tensors.pk', 'wb') as file:
    pickle.dump(pairs_val_tensor, file)

In [33]:
symb2index

{'<pad>': 0,
 '<sos>': 1,
 '<eos>': 2,
 'M': 3,
 'A': 4,
 'F': 5,
 'K': 6,
 'Y': 7,
 'L': 8,
 'P': 9,
 'I': 10,
 'G': 11,
 'S': 12,
 'N': 13,
 'E': 14,
 'R': 15,
 'Q': 16,
 'V': 17,
 'T': 18,
 'H': 19,
 'C': 20,
 'D': 21,
 'W': 22,
 'c': 23,
 '1': 24,
 '(': 25,
 ')': 26,
 '=': 27,
 'O': 28,
 '2': 29,
 '[': 30,
 'n': 31,
 ']': 32,
 '-': 33,
 '3': 34,
 '4': 35,
 '@': 36,
 '5': 37,
 '+': 38,
 'l': 39,
 'B': 40,
 'r': 41,
 '#': 42,
 '\\': 43,
 '/': 44,
 's': 45,
 'o': 46,
 '6': 47,
 'b': 48,
 '.': 49,
 'u': 50,
 'i': 51,
 'e': 52,
 'a': 53,
 '7': 54,
 '8': 55,
 '9': 56,
 'd': 57,
 '*': 58,
 'Z': 59,
 'g': 60,
 '%': 61,
 '0': 62}

In [34]:
with open('symbole_to_index_encoding.pk', 'wb') as file:
    pickle.dump(symb2index, file)
    
with open('index_to_symbol_decoding.pk', 'wb') as file:
    pickle.dump(index2symb, file)

In [35]:
test = pd.read_pickle('pairs_train_tensors.pk')

In [36]:
test[0][1].device

device(type='cpu')